<a href="https://colab.research.google.com/github/Hainguyendangduc/DeepLearning_PJ/blob/main/LSTM_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip

--2022-05-26 10:08:02--  https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84188 (82K) [application/x-httpd-php]
Saving to: ‘sentiment labelled sentences.zip.1’

sentiment labelled  100%[===================>]  82.21K  --.-KB/s    in 0.1s    

2022-05-26 10:08:02 (559 KB/s) - ‘sentiment labelled sentences.zip.1’ saved [84188/84188]



In [ ]:
!unzip sentences.zip

unzip:  cannot find or open sentences.zip, sentences.zip.zip or sentences.zip.ZIP.


In [ ]:
X = [] # input text
y = [] # label

f = open(file='a/amazon_cells_labelled.txt', mode='r')
for line in f.readlines():
  line = line.strip()
  line = line.split("\t")
  y.append(int(line[-1]))
  X.append(line[0].strip())
f.close()

f = open('a/yelp_labelled.txt', 'r')
for line in f.readlines():
  line = line.strip()
  line = line.split("\t")
  y.append(int(line[-1]))
  X.append(line[0].strip())
f.close()

f = open('a/imdb_labelled.txt', 'r')
for line in f.readlines():
  line = line.strip()
  line = line.split("\t")
  y.append(int(line[-1]))
  X.append(line[0].strip())
f.close()


print(X[0])
print(len(X))
print(y[0])
print(len(y))


FileNotFoundError: ignored

In [ ]:
from matplotlib import pyplot as plt
print(X[0])
print(len(X), len(y))
plt.hist(y, bins=20)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=0)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(X)

X_seq_train = tokenizer.texts_to_sequences(X_train)
X_seq_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print(X_seq_train[0])
print(tokenizer.sequences_to_texts([[432, 47]]))
print(X_train[0])

In [ ]:
for i in X_seq_train[0]:
   print(f"{i} -----> {tokenizer.index_word[i]}")

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = max([len(t) for t in X_seq_train] + [len(t) for t in X_seq_test])
print(max_length)
X_pad_train = pad_sequences(sequences=X_seq_train, 
                              maxlen=max_length, 
                              padding='post')

print(X_pad_train.shape)

X_pad_test = pad_sequences(sequences=X_seq_test, 
                              maxlen=max_length,     
                              padding='post')
print(X_pad_test.shape)

In [ ]:
X_pad_train[1]

In [ ]:
# ML: X_pad ->  sentiment 
from tensorflow.keras.layers import Embedding, Input, Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

vocab_size = 2000
p = 0.1

inp = Input(shape=(max_length))
x = Embedding(vocab_size, 128, input_length=max_length)(inp)
x = Dropout(p)(x)
x = Conv1D(filters=32, kernel_size=3, 
           padding='same', activation='relu')(x)

x = Dropout(p)(x)
x = Conv1D(filters=16, kernel_size=3, 
           padding='same', activation='relu')(x)

# collect features: poolling??? -> inovations 
# dãn ma trận 59 x 16 -> 944-vector
#x = Flatten()(x)

# Trung bình hoá các feature của chuỗi
x = K.mean(x, keepdims=False, axis=1)

# Fully connected layer for classification
x = Dense(units=16, activation='relu')(x)
x = Dropout(p)(x)
x = Dense(units=1, activation='sigmoid')(x) # xác suất để x là positive 

model = Model(inputs=inp, outputs=x)
model.summary()

# compile model -> optimizer, loss, evaluation metrics
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['acc'])

In [ ]:
y = np.array(y)
model.fit(X_pad_train, np.array(y_train), 
          epochs=10, batch_size=64, validation_split=0.2)

In [ ]:
y_pred = model.predict(X_pad_test)
y_pred[y_pred>0.5] = 1
y_pred[y_pred<=0.5] = 0

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
def predict(seq, model):
  test_text = [seq]
  X_test = tokenizer.texts_to_sequences(test_text)
  X_test_padd = pad_sequences(sequences=X_test, maxlen=max_length, padding='post')

  p = model.predict(X_test_padd)[0, 0]

  if p > 0.5: 
    return "Positive", p
  else:
    return "Negatvie", p

seq = "it is terrible today"

predict(seq, model)

In [ ]:
y[-1]

In [ ]:
import numpy as np

seq = np.array([[0, 1, 0], 
                [1, 0,  0], 
                [0, 0, 1]])

Wh = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]) # 1 layer -> #row = #input, #col = #output
Wx = np.array([[0.1, 0.2, 0.3], [.4, .5, .6], [.7, .8, .9]])

ho = np.array([0, 0, 0])

g = lambda z: np.tanh(z)
for i in range(3):
  h1 = g(np.dot(ho, Wh) + np.dot(seq[i], Wx))
  print(h1)

  ho = h1



In [ ]:
# text -> RNN -> using final state as the features of text
# seq = "the phone was broken on shipping, terrible" -> RNN -> final state ->FC layer -> sentiment

from tensorflow.keras.layers import LSTM

inp = Input(shape=(max_length))

x = Embedding(vocab_size, 128, input_length=200)(inp)
x, h, c = LSTM(units=128, return_state=True, return_sequences=True)(x)

x = Conv1D(filters=32, kernel_size=3, 
           padding='same', activation='relu')(x)

x = K.mean(x, keepdims=False, axis=1)
y = Dense(units=64, activation='relu')(x)
y = Dense(units=1, activation='sigmoid')(y)

model = Model(inputs=inp, outputs=y)

model.summary()

# compile model -> optimizer, loss, evaluation metrics
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['acc'])

In [ ]:
model.fit(X_pad_train, np.array(y_train), 
          epochs=20, batch_size=64, validation_split=0.2)

In [ ]:
def predict(seq, model):
  test_text = [seq]
  X_test = tokenizer.texts_to_sequences(test_text)
  X_test_padd = pad_sequences(sequences=X_test, maxlen=max_length, padding='post')

  p = model.predict(X_test_padd)[0, 0]

  if p > 0.5: 
    return "Positive", p
  else:
    return "Negatvie", p

seq = "the phone was broken on shipping"

predict(seq, model)

In [ ]:
y_pred = model.predict(X_pad_test)
y_pred[y_pred>0.5] = 1
y_pred[y_pred<=0.5] = 0

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
# Attention
# text -> RNN -> using final state as the features of text
# seq = "the phone was broken on shipping, terrible" -> RNN -> final state ->FC layer -> sentiment

from tensorflow.keras.layers import LSTM, Softmax


inp = Input(shape=(max_length))

x = Embedding(vocab_size, 128, input_length=200)(inp)
x, h, c = LSTM(units=16, return_state=True, return_sequences=True)(x)
x = Conv1D(filters=32, kernel_size=5, 
           padding='same', activation='relu')(x)

# Attention part
att = Conv1D(filters=1, kernel_size=1, 
           padding='same', activation='relu')(x)
att = Softmax(axis=1)(att)

x = K.sum(att * x, axis=1)
y = Dense(units=64, activation='relu')(x)
y = Dense(units=1, activation='sigmoid')(y)

model = Model(inputs=inp, outputs=y)

model.summary()

# compile model -> optimizer, loss, evaluation metrics
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['acc'])

In [ ]:
model.fit(X_pad_train, np.array(y_train), 
          epochs=20, batch_size=64, validation_split=0.2)

In [ ]:
# Bidirectioal LSTM
# text -> RNN -> using final state as the features of text
# seq = "the phone was broken on shipping, terrible" -> RNN -> final state ->FC layer -> sentiment

from tensorflow.keras.layers import LSTM, Softmax, Bidirectional


inp = Input(shape=(max_length))

x = Embedding(vocab_size, 16)(inp)
x = Bidirectional(LSTM(units=16, return_sequences=True))(x)

x = Conv1D(filters=32, kernel_size=5, 
           padding='same', activation='relu')(x)

# Attention part
att = Conv1D(filters=1, kernel_size=1, 
           padding='same', activation='relu')(x)
att = Softmax(axis=1)(att)

x = K.sum(att * x, axis=1)
y = Dense(units=64, activation='relu')(x)
y = Dense(units=1, activation='sigmoid')(y)

model = Model(inputs=inp, outputs=y)

model.summary()

# compile model -> optimizer, loss, evaluation metrics
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['acc'])

In [ ]:
model.fit(X_pad_train, np.array(y_train), 
          epochs=20, batch_size=64, validation_split=0.2)